# Assignment - II

## Group Member Names

- Akhila Guttikonda
- Sai Bhargav Tetali
- Shubhada Kapre
- Soumith Reddy Palreddy
- Srividya Rayaprolu

## Task A

In [3]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import re
%pylab inline

from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [26]:
#run on collab only
#!pip install selenium
#!apt-get update 
#!apt install chromium-chromedriver

from selenium import webdriver
from selenium.webdriver.common.by import By
chrome_options = webdriver.ChromeOptions()

#run on collab only
#chrome_options.add_argument('--headless')
#chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--disable-dev-shm-usage')

path = '/Users/saibhargavtetali/chromedriver'
driver = webdriver.Chrome(path,options=chrome_options)
url_list = ['https://www.beeradvocate.com/beer/popular/','https://www.beeradvocate.com/beer/worst/',\
            'https://www.beeradvocate.com/beer/top-rated/']

# the following code is to extract all the links of beers present in the pages 'popular', 'worst' and 'top-rated'
correct_links = []
for a in url_list:
    driver.get(a)
    links_list = []
    links = driver.find_elements(By.TAG_NAME,'a')
    for lnk in links:
       # get_attribute() to get all href
       links_list.append(lnk.get_attribute('href'))
        
    for i in links_list:
        if "/profile/" in str(i) and i.count('/') == 7:
            correct_links.append(i)
correct_links_nd = list(set(correct_links))

driver.quit()
len(correct_links_nd)

719

We are going to scrape reviews of **719** beers from the beeradvocate website

In [27]:
# the following code is going to open each beer page and scrape reviews of the beer
comments = []
product_name = []
path = '/Users/saibhargavtetali/chromedriver'
driver = webdriver.Chrome(path,options=chrome_options)

for j in range(len(correct_links_nd)):
    driver.get(str(correct_links_nd[j]))
    beer = driver.find_element(By.CLASS_NAME, 'titleBar').text.split('\n')[0]
    review = driver.find_elements(By.CLASS_NAME,'user-comment')
    for i in review:
        comment = i.find_element(By.ID,'rating_fullview_content_2').text
        comments.append(comment)
        product_name.append(beer)
driver.quit()

In [28]:
df = pd.DataFrame(columns = ["product_name", "user_rating","raw_comments", "product_review"])
df['product_name'] = product_name
df['raw_comments'] = comments
string = 'Rated: '
df['user_rating'] = df['raw_comments'].map(lambda x: x.split('\n')[2].split('/')[0] if(string not in x) else np.nan)
df['product_review'] = df['raw_comments'].map(lambda x: (' '.join(x.split('\n')[4:-1])).lower() if ('Rated: ' not in x) else np.nan)
df.drop(columns = ['raw_comments'], inplace = True)
df.dropna(inplace = True)
df.reset_index(drop = True, inplace = True)
df['product_review'] = df['product_review'].map(lambda x: re.sub(r'[^\w\s]', '', x.replace("\n", " ")))

stop_words = set(stopwords.words('english'))

# this function is to create a new column of text by removing stopwords. A new column of reviews without stopwords
# is being created to get the word frequencies in the next task for recognizing beer attributes
def remstopwords(x):
    text_tokens = word_tokenize(str(x))
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    return (" ").join(tokens_without_sw)

df['product_review_nostopw'] = df['product_review'].map(remstopwords)
# saving the scraped data to csv so that we don't have to repeat the above process again
df.to_csv('Comments.csv')
df

,product_name,user_rating,product_review,product_review_nostopw
0,Vanilla Porter,3.35,poured from a 12oz can best by january 29th 20...,poured 12oz january 29th 2023 poured khaki 2 f...
1,Vanilla Porter,2.8,look dark red no head smell i cant really des...,dark red head smell describe smells taste wate...
2,Vanilla Porter,3.55,pour a very effervescent dark copper with slig...,effervescent dark copper slightly finger head ...
3,Vanilla Porter,3.98,great vanilla taste full of flavor and is not ...,great vanilla taste full flavor heavy definite...
4,Vanilla Porter,3.53,from a can caramel color with a thin layer of ...,caramel color thin layer colored head smells r...
...,...,...,...,...
9155,Carlton Dry,1.85,sulfur vegetable on the nose softer carbonatio...,sulfur vegetable nose softer carbonation expec...
9156,Carlton Dry,1.68,appearance 20 pours a finger of white head th...,appearance 20 pours finger white head quickly ...
9157,Carlton Dry,1.42,pours a standard beer gold the colour of beer ...,pours standard beer gold colour beer carbonati...
9158,Carlton Dry,1.83,tiny generic lager head with slight lacing cry...,tiny generic lager head slight lacing crystal ...


In [4]:
# reading in the previously saved reviews data
df = pd.read_csv('Comments.csv')
df.drop(columns = ['Unnamed: 0'], inplace = True)
df

,product_name,user_rating,product_review,product_review_nostopw
0,Vanilla Porter,3.35,poured from a 12oz can best by january 29th 20...,poured 12oz january 29th 2023 poured khaki 2 f...
1,Vanilla Porter,2.80,look dark red no head smell i cant really des...,dark red head smell describe smells taste wate...
2,Vanilla Porter,3.55,pour a very effervescent dark copper with slig...,effervescent dark copper slightly finger head ...
3,Vanilla Porter,3.98,great vanilla taste full of flavor and is not ...,great vanilla taste full flavor heavy definite...
4,Vanilla Porter,3.53,from a can caramel color with a thin layer of ...,caramel color thin layer colored head smells r...
...,...,...,...,...
9155,Carlton Dry,1.85,sulfur vegetable on the nose softer carbonatio...,sulfur vegetable nose softer carbonation expec...
9156,Carlton Dry,1.68,appearance 20 pours a finger of white head th...,appearance 20 pours finger white head quickly ...
9157,Carlton Dry,1.42,pours a standard beer gold the colour of beer ...,pours standard beer gold colour beer carbonati...
9158,Carlton Dry,1.83,tiny generic lager head with slight lacing cry...,tiny generic lager head slight lacing crystal ...


## Task B

In [5]:
# this function is to calculate the unique frequencies of words(excluding stopwords) in the reviews

def unique_freq(Series):
    
    y = Series.map(lambda x: list(set(str(x).lstrip('"').rstrip('"').split())))

    wordlists_list = list(y)
    freq_dict = {}
    for i in range (len(wordlists_list)):
        for j in range(len(wordlists_list[i])):
            if wordlists_list[i][j] in freq_dict:
                freq_dict[wordlists_list[i][j]] = freq_dict[wordlists_list[i][j]] + 1
            else:
                freq_dict[wordlists_list[i][j]] = 1

    uniq_freq = pd.DataFrame(list(freq_dict.items()),columns = ['word','count']).set_index('word')
    return uniq_freq

df_unique_freq = unique_freq(df['product_review_nostopw']).sort_values(by = 'count', ascending = False)
df_unique_freq.to_csv('Unique_Word_freq.csv')
df_unique_freq

,count
word,
head,4445
beer,4379
taste,3902
light,2522
carbonation,2278
...,...
herc,1
121024,1
t0tal,1


## Task C

The following are the top 10 beer attributes found in the data:
- carbonation 
- sweet 
- malt
- bitterness
- dark
- smooth
- dry
- chocolate
- citrus
- caramel

Note: For the coming tasks we are taking the user inputed attributes from the excel file 'product_attributes.xlsx'. We are assuming user has given the top 3 attributes 'carbonation','sweet' and 'malt'. For other attributes we simply need to change the excel file and run the code again.

In [6]:
# reading in the attributes given as input by the user. For our analysis we will be using the top 3 attributes
# 'carbonation', 'sweet' and 'malt'

df_attributes = pd.read_excel('product_attributes.xlsx')
df_attributes['product_attributes'] = df_attributes['product_attributes'].map(lambda x: x.lower())
attributes = list(df_attributes['product_attributes'])
attributes

['carbonation', 'sweet', 'malt']

In [7]:
# the below function is to calculate the term frequencies

def term_freq(x):
    tf_list = []
    for i in attributes:
        tf_list.append(str(x).count(i))
        # use below code for term frequency if duplicate values are not to be considered
        #if str(x).count(i)>0:
            #tf_list.append(1)
        #else:
            #tf_list.append(0)
        #use below code for term frequency if normalization is to be considered
        #tf_list.append(str(x).count(i)/len(str(x).split()))
    return tf_list
df['term_frequency'] = df['product_review'].map(term_freq)

# calculating the IDF(inverse document frequency) of the attributes 
log_idf = np.zeros(len(attributes))
for i in range(len(attributes)):
    # if idf is to be considered
    log_idf[i] = np.log10(len(df)/(df['product_review'].str.contains(attributes[i], na=False).sum()))
    # else use
    #log_idf[i] = 1

#q_idf = log_idf/len(attributes) 

def similarity(x):
    y = np.zeros(len(attributes))
    for i in range(len(attributes)):
        y[i] = x[i]*(log_idf[i])
    similarity = np.dot(y,log_idf)/(norm(y)*norm(log_idf))
    return similarity
df['similarity_score'] = df['term_frequency'].map(similarity).fillna(0)
df.drop(columns = ['term_frequency'], inplace = True)
df[df['similarity_score']>0]

/var/folders/_b/x_7_szcn2ddbs02_yffgqgyr0000gn/T/ipykernel_7140/3463288935.py:31: RuntimeWarning: invalid value encountered in double_scalars
  similarity = np.dot(y,log_idf)/(norm(y)*norm(log_idf))


,product_name,user_rating,product_review,product_review_nostopw,similarity_score
0,Vanilla Porter,3.35,poured from a 12oz can best by january 29th 20...,poured 12oz january 29th 2023 poured khaki 2 f...,1.000000
2,Vanilla Porter,3.55,pour a very effervescent dark copper with slig...,effervescent dark copper slightly finger head ...,0.538703
4,Vanilla Porter,3.53,from a can caramel color with a thin layer of ...,caramel color thin layer colored head smells r...,0.530388
6,Vanilla Porter,2.89,almost no scent when the cap comes off this po...,scent cap porter developed nice tannish head q...,0.654590
7,Vanilla Porter,2.00,look dark brown to black with a quickly dissip...,dark brown black quickly dissipating head mini...,0.654590
...,...,...,...,...,...
9154,Carlton Dry,2.04,a served a light yellow color with a small hea...,served light yellow color small head fades qui...,0.717567
9155,Carlton Dry,1.85,sulfur vegetable on the nose softer carbonatio...,sulfur vegetable nose softer carbonation expec...,0.847755
9156,Carlton Dry,1.68,appearance 20 pours a finger of white head th...,appearance 20 pours finger white head quickly ...,0.538703
9157,Carlton Dry,1.42,pours a standard beer gold the colour of beer ...,pours standard beer gold colour beer carbonati...,0.943587


## Task D

In [8]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sentiment_obj = SentimentIntensityAnalyzer()

def sentiment_score(x):
    return sentiment_obj.polarity_scores(str(x))['compound']

df['sentiment_score'] = df['product_review'].map(sentiment_score)
df

,product_name,user_rating,product_review,product_review_nostopw,similarity_score,sentiment_score
0,Vanilla Porter,3.35,poured from a 12oz can best by january 29th 20...,poured 12oz january 29th 2023 poured khaki 2 f...,1.000000,0.9781
1,Vanilla Porter,2.80,look dark red no head smell i cant really des...,dark red head smell describe smells taste wate...,0.000000,0.8462
2,Vanilla Porter,3.55,pour a very effervescent dark copper with slig...,effervescent dark copper slightly finger head ...,0.538703,0.6486
3,Vanilla Porter,3.98,great vanilla taste full of flavor and is not ...,great vanilla taste full flavor heavy definite...,0.000000,0.8519
4,Vanilla Porter,3.53,from a can caramel color with a thin layer of ...,caramel color thin layer colored head smells r...,0.530388,0.5719
...,...,...,...,...,...,...
9155,Carlton Dry,1.85,sulfur vegetable on the nose softer carbonatio...,sulfur vegetable nose softer carbonation expec...,0.847755,0.5023
9156,Carlton Dry,1.68,appearance 20 pours a finger of white head th...,appearance 20 pours finger white head quickly ...,0.538703,-0.9575
9157,Carlton Dry,1.42,pours a standard beer gold the colour of beer ...,pours standard beer gold colour beer carbonati...,0.943587,-0.9196
9158,Carlton Dry,1.83,tiny generic lager head with slight lacing cry...,tiny generic lager head slight lacing crystal ...,0.842496,0.1885


## Task E

In [9]:
df['total_score'] = df['similarity_score']*df['sentiment_score']
df

,product_name,user_rating,product_review,product_review_nostopw,similarity_score,sentiment_score,total_score
0,Vanilla Porter,3.35,poured from a 12oz can best by january 29th 20...,poured 12oz january 29th 2023 poured khaki 2 f...,1.000000,0.9781,0.978100
1,Vanilla Porter,2.80,look dark red no head smell i cant really des...,dark red head smell describe smells taste wate...,0.000000,0.8462,0.000000
2,Vanilla Porter,3.55,pour a very effervescent dark copper with slig...,effervescent dark copper slightly finger head ...,0.538703,0.6486,0.349403
3,Vanilla Porter,3.98,great vanilla taste full of flavor and is not ...,great vanilla taste full flavor heavy definite...,0.000000,0.8519,0.000000
4,Vanilla Porter,3.53,from a can caramel color with a thin layer of ...,caramel color thin layer colored head smells r...,0.530388,0.5719,0.303329
...,...,...,...,...,...,...,...
9155,Carlton Dry,1.85,sulfur vegetable on the nose softer carbonatio...,sulfur vegetable nose softer carbonation expec...,0.847755,0.5023,0.425827
9156,Carlton Dry,1.68,appearance 20 pours a finger of white head th...,appearance 20 pours finger white head quickly ...,0.538703,-0.9575,-0.515808
9157,Carlton Dry,1.42,pours a standard beer gold the colour of beer ...,pours standard beer gold colour beer carbonati...,0.943587,-0.9196,-0.867723
9158,Carlton Dry,1.83,tiny generic lager head with slight lacing cry...,tiny generic lager head slight lacing crystal ...,0.842496,0.1885,0.158810


We took the product of similarity score and sentiment score to keep the negative/positive values intact.

In [11]:
beer_bow = df.groupby('product_name')['total_score'].mean().sort_values(ascending = False)
top3_beer_bow = beer_bow[:3].index.values
top3_beer_bow

array(['Ann', 'Duchesse De Bourgogne', 'Old Chub'], dtype=object)

For the attributes *carbonation*, *sweet* and *malt* the recommender built with the vanilla bag of words is suggesting **Ann**, **Duchesse De Bourgogne** and **Old Chub**.  

Note: The top 3 suggestions are based on the average total_scores of each of the beers

## Task F

In [12]:
# importing spacy package with medium sized pre-trained word vectors to calculate spacy_similarity scores 

import spacy

nlp = spacy.load('en_core_web_md')

def spacy_similarity(x):
    y = nlp(str(x))
    att = nlp(' '.join(attributes))
    return y.similarity(att)

df['spacy_similarity'] = df['product_review'].map(spacy_similarity)
df

/var/folders/_b/x_7_szcn2ddbs02_yffgqgyr0000gn/T/ipykernel_7140/2766996903.py:10: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return y.similarity(att)


,product_name,user_rating,product_review,product_review_nostopw,similarity_score,sentiment_score,total_score,spacy_similarity
0,Vanilla Porter,3.35,poured from a 12oz can best by january 29th 20...,poured 12oz january 29th 2023 poured khaki 2 f...,1.000000,0.9781,0.978100,0.599463
1,Vanilla Porter,2.80,look dark red no head smell i cant really des...,dark red head smell describe smells taste wate...,0.000000,0.8462,0.000000,0.564001
2,Vanilla Porter,3.55,pour a very effervescent dark copper with slig...,effervescent dark copper slightly finger head ...,0.538703,0.6486,0.349403,0.565585
3,Vanilla Porter,3.98,great vanilla taste full of flavor and is not ...,great vanilla taste full flavor heavy definite...,0.000000,0.8519,0.000000,0.581631
4,Vanilla Porter,3.53,from a can caramel color with a thin layer of ...,caramel color thin layer colored head smells r...,0.530388,0.5719,0.303329,0.645449
...,...,...,...,...,...,...,...,...
9155,Carlton Dry,1.85,sulfur vegetable on the nose softer carbonatio...,sulfur vegetable nose softer carbonation expec...,0.847755,0.5023,0.425827,0.598142
9156,Carlton Dry,1.68,appearance 20 pours a finger of white head th...,appearance 20 pours finger white head quickly ...,0.538703,-0.9575,-0.515808,0.595241
9157,Carlton Dry,1.42,pours a standard beer gold the colour of beer ...,pours standard beer gold colour beer carbonati...,0.943587,-0.9196,-0.867723,0.577959
9158,Carlton Dry,1.83,tiny generic lager head with slight lacing cry...,tiny generic lager head slight lacing crystal ...,0.842496,0.1885,0.158810,0.689953


In [13]:
# multiplying spacy_score similarity score with sentiment scores to get tota_score_spacy score to replace the 
# total_score calculated from the bag of words model

df['total_score_spacy'] = df['spacy_similarity']*df['sentiment_score']
df

,product_name,user_rating,product_review,product_review_nostopw,similarity_score,sentiment_score,total_score,spacy_similarity,total_score_spacy
0,Vanilla Porter,3.35,poured from a 12oz can best by january 29th 20...,poured 12oz january 29th 2023 poured khaki 2 f...,1.000000,0.9781,0.978100,0.599463,0.586335
1,Vanilla Porter,2.80,look dark red no head smell i cant really des...,dark red head smell describe smells taste wate...,0.000000,0.8462,0.000000,0.564001,0.477257
2,Vanilla Porter,3.55,pour a very effervescent dark copper with slig...,effervescent dark copper slightly finger head ...,0.538703,0.6486,0.349403,0.565585,0.366838
3,Vanilla Porter,3.98,great vanilla taste full of flavor and is not ...,great vanilla taste full flavor heavy definite...,0.000000,0.8519,0.000000,0.581631,0.495492
4,Vanilla Porter,3.53,from a can caramel color with a thin layer of ...,caramel color thin layer colored head smells r...,0.530388,0.5719,0.303329,0.645449,0.369132
...,...,...,...,...,...,...,...,...,...
9155,Carlton Dry,1.85,sulfur vegetable on the nose softer carbonatio...,sulfur vegetable nose softer carbonation expec...,0.847755,0.5023,0.425827,0.598142,0.300447
9156,Carlton Dry,1.68,appearance 20 pours a finger of white head th...,appearance 20 pours finger white head quickly ...,0.538703,-0.9575,-0.515808,0.595241,-0.569943
9157,Carlton Dry,1.42,pours a standard beer gold the colour of beer ...,pours standard beer gold colour beer carbonati...,0.943587,-0.9196,-0.867723,0.577959,-0.531491
9158,Carlton Dry,1.83,tiny generic lager head with slight lacing cry...,tiny generic lager head slight lacing crystal ...,0.842496,0.1885,0.158810,0.689953,0.130056


In [14]:
beer_spacy = df.groupby('product_name')['total_score_spacy'].mean().sort_values(ascending = False)
top3_beer_spacy = beer_spacy.index.values[:3]
top3_beer_spacy

array(['Triple Shot', 'Flora Plum',
       'Double Dry Hopped Double Mosaic Daydream'], dtype=object)

For the attributes *carbonation*, *sweet* and *malt* the recommender built with the spacy package is suggesting **Triple Shot**, **Flora Plum** and **Double Dry Hopped Double Mosaic Daydream**

Note: The top 3 suggestions are based on the average total_score_spacy of each of the beers

Recommendations obtained from word embeddings approach is completely different from the bag of words approach. Word embeddings has a huge number of words with similar meanings in the vector which gives a generalized analysis while the bag of words is very specific to the targeted features – in our case they’re *carbonation*, *sweet* and *malt*.

In [15]:
def review_percentage(x):
    review_percentage = {}
    for i in x:
        df_mask = df[df['product_name'] == i]
        att_percentage_list = []
        for j in attributes:
            att_percentage_list.append((df_mask['product_review'].str.contains(j,na=False)).sum()/len(df_mask))
        review_percentage[i] = att_percentage_list
    return review_percentage
pd.DataFrame(review_percentage(top3_beer_bow)).set_axis(attributes).T*100

,carbonation,sweet,malt
Ann,75.000000,75.000000,0.000000
Duchesse De Bourgogne,64.285714,85.714286,28.571429
Old Chub,40.000000,86.666667,93.333333


The above numbers represent the % of reviews containing the respective attribute. For example, 40% of the Old Chub reviews have the word carbonation in them. The above beers are the top 3 recommended beers using Bag of Words approach

In [16]:
pd.DataFrame(review_percentage(top3_beer_spacy)).set_axis(attributes).T*100

,carbonation,sweet,malt
Triple Shot,0.0,100.0,25.0
Flora Plum,20.0,20.0,20.0
Double Dry Hopped Double Mosaic Daydream,37.5,62.5,25.0


The above beers are the top 3 recommended beers using Word Embeddings approach

We can see that Bag of Words recommended beers have higher % of reviews talking about the attriutes compared to those coming from Word Embeddings approach

## Task G

In [17]:
top_3 = df.groupby('product_name')['user_rating'].mean().sort_values(ascending = False)[:3].index.values
top_3

array(['The Adjunct Trail - Bourbon Barrel-Aged',
       'Twice the Daily Serving: Raspberry', 'Art'], dtype=object)

**The Adjunct Trail - Bourbon Barrel-Aged**, **Twice the Daily Serving: Raspberry** and **Art** are the top 3 beers in terms of ratings

In [18]:
pd.DataFrame(review_percentage(top_3)).set_axis(attributes).T*100

,carbonation,sweet,malt
The Adjunct Trail - Bourbon Barrel-Aged,0.0,0.0,0.000000
Twice the Daily Serving: Raspberry,0.0,50.0,16.666667
Art,0.0,0.0,0.000000


We can see that the top-3 rated beers do not have much to do with the attributes we are looking for by looking at the % of reviews containing the attributes

In [19]:
beer_bow[beer_bow.index.isin(np.concatenate((top3_beer_bow, top3_beer_spacy,top_3), axis=0, out=None))]

product_name
Ann                                         0.659717
Duchesse De Bourgogne                       0.659466
Old Chub                                    0.656538
Triple Shot                                 0.566816
Double Dry Hopped Double Mosaic Daydream    0.445272
Flora Plum                                  0.273815
Twice the Daily Serving: Raspberry          0.258048
Art                                         0.000000
The Adjunct Trail - Bourbon Barrel-Aged     0.000000
Name: total_score, dtype: float64

In the above list the numbers are the total_scores from bag of words approach. We can see that 2 of the top 3 rated beers do not even have the attriutes in their reviews.

In [20]:
beer_spacy[beer_spacy.index.isin(np.concatenate((top3_beer_spacy, top3_beer_bow,top_3), axis=0, out=None))]

product_name
Triple Shot                                 0.606792
Flora Plum                                  0.584741
Double Dry Hopped Double Mosaic Daydream    0.574405
Ann                                         0.565550
Old Chub                                    0.538434
Duchesse De Bourgogne                       0.536077
Twice the Daily Serving: Raspberry          0.460784
The Adjunct Trail - Bourbon Barrel-Aged     0.458123
Art                                         0.182889
Name: total_score_spacy, dtype: float64

In the above list the numbers are the total_score_spacy from word embeddings approach. We can see that top 3 rated beers have improved scores but we have already seen in the previous task that the Word embeddings approach might not be an optimal way to recommend beers based on the given attributes.

We cannot recommend the beers that we got based on the ratings given by the users as they are based on their individual requirements and preferences.

## Task H

In [21]:
top4_attributes = ['carbonation', 'sweet', 'malt', 'bitterness'] #determined from word frequency analysis

beer_list = ['IPA', 'Somewhere, Something Incredible Is Waiting To Be Known', 'Bud Light Lime', 'Heineken Lager Beer',\
            'A Deal With The Devil - Triple Oak-Aged', 'Taj Mahal Premium Lager', 'Budweiser Select', 'Old German',\
             'Zombie Dust', 'Maximus']

def lift(string, string_list,dataf,beer_column,review_column,freq_series):
    nrows = float(len(dataf))
    lift_list = []
    string_mask = dataf[beer_column] == string
    string_list_n = float(string_mask.sum())
    for items in string_list:
        string_n = float(freq_series.loc[items])
        mel_count = float(dataf[string_mask][review_column].str.contains(items).sum())
        lift = (nrows * mel_count)/(string_n * string_list_n)
        lift_list.append(lift)
    
    return lift_list

lift_dict = {}
for i in range(len(beer_list)):
    lift_dict[beer_list[i]] = lift(beer_list[i], top4_attributes,df,'product_name','product_review',\
                                         df_unique_freq)

beers_lift = pd.DataFrame.from_dict(lift_dict)
beers_lift.index = top4_attributes
beers_lift.T

,carbonation,sweet,malt,bitterness
IPA,0.957398,1.358181,2.201870,1.599022
"Somewhere, Something Incredible Is Waiting To Be Known",2.010536,4.503441,1.155982,0.000000
Bud Light Lime,0.861658,0.321674,0.990842,0.513971
Heineken Lager Beer,1.005268,0.562930,1.444977,0.000000
A Deal With The Devil - Triple Oak-Aged,0.670179,1.501147,0.770655,0.000000
Taj Mahal Premium Lager,1.005268,2.533186,2.311964,0.000000
Budweiser Select,0.473067,0.794725,0.815987,0.000000
Old German,0.618626,1.385674,1.778434,1.107016
Zombie Dust,0.957398,1.286698,1.541309,1.027943
Maximus,0.765918,1.715597,2.862431,1.370591


The above values are the lift values between the beers and attributes.

In [22]:
lift_cosine = {}
for keys1 in lift_dict:
    keys1_lift_list = []
    for keys2 in lift_dict:
            #keys1_lift_list.append(np.dot(np.array(lift_dict[keys1]),np.array(lift_dict[keys2]))/(norm(np.array(lift_dict[keys1]))*norm(np.array(lift_dict[keys2]))))
            keys1_lift_list.append(np.linalg.norm(np.array(lift_dict[keys1]) - np.array(lift_dict[keys2])))
    lift_cosine[keys1] = keys1_lift_list

lift_cosine
lift_cosine_df = pd.DataFrame.from_dict(lift_cosine)
lift_cosine_df.index = beer_list
lift_cosine_df

,IPA,"Somewhere, Something Incredible Is Waiting To Be Known",Bud Light Lime,Heineken Lager Beer,A Deal With The Devil - Triple Oak-Aged,Taj Mahal Premium Lager,Budweiser Select,Old German,Zombie Dust,Maximus
IPA,0.000000,3.827861,1.930657,1.940225,2.169835,1.987944,2.242678,0.732729,0.876118,0.808042
"Somewhere, Something Incredible Is Waiting To Be Known",3.827861,0.000000,4.370187,4.076973,3.310409,2.495749,4.029142,3.642913,3.558326,3.756546
Bud Light Lime,1.930657,4.370187,0.000000,0.741103,1.319269,2.630767,0.818242,1.470769,1.227851,2.487736
Heineken Lager Beer,1.940225,4.076973,0.741103,0.000000,1.203016,2.152574,0.855918,1.470740,1.261775,2.296437
A Deal With The Devil - Triple Oak-Aged,2.169835,3.310409,1.319269,1.203016,0.000000,1.884946,0.734806,1.502363,1.333814,2.511813
Taj Mahal Premium Lager,1.987944,2.495749,2.630767,2.152574,1.884946,0.000000,2.354449,1.725228,1.790698,1.705073
Budweiser Select,2.242678,4.029142,0.818242,0.855918,0.734806,2.354449,0.000000,1.588142,1.435051,2.645793
Old German,0.732729,3.642913,1.470769,1.470740,1.502363,1.725228,1.588142,0.000000,0.432485,1.172632
Zombie Dust,0.876118,3.558326,1.227851,1.261775,1.333814,1.790698,1.435051,0.432485,0.000000,1.443395
Maximus,0.808042,3.756546,2.487736,2.296437,2.511813,1.705073,2.645793,1.172632,1.443395,0.000000


We calculated the euclidian distance between beer vectors using the lift values of a beer for each attribute. The above values are the euclidian distances of the attribute lift values.

For **IPA** the most similar beer in the 10 beers we selected is **Old German**. But for **Old German** the closest beer is **Zombie Dust**. If we compare the lift values in the previous table Old German and Zombie Dust have similar lift values in the respective attributes.